In [ ]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 7.9 MB/s 
     |████████████████████████████████| 636 kB 58.2 MB/s 
     |████████████████████████████████| 5.2 MB 39.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 85 kB 4.9 MB/s 
     |████████████████████████████████| 88 kB 7.8 MB/s 
     |████████████████████████████████| 5.5 MB 54.2 MB/s 
     |████████████████████████████████| 1.3 MB 55.0 MB/s 
     |████████████████████████████████| 182 kB 66.4 MB/s 
     |████████████████████████████████| 7.6 MB 34.3 MB/s 
     |████████████████████████████████| 1.1 MB 51.9 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp37-cp37m-linux_x86_64.whl size=2340694 sha256=94b4bf25da4c4670db4b4f0b9c85a5e1bbfc67fdd46391b80f1b242df2697030
  Stored in directory: /root/.cache

In [ ]:
!pip install bertopic[visualization]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 30.97 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 465 kB 66.3 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-11-21 14:08:29--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22e9:9f55, 2406:da00:ff00::3403:4be7, ...
Connecting to bitbucket.org (b

필요한 것들 임포트

In [ ]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from bertopic import BERTopic
import pandas as pd
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cd /content

/content


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/구미시 뉴스기사 분류/NewsResult_20180101-20221120.xlsx')
df.head()

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부
0,1.500501e+06,20221120,대구일보,류성욱,이영 중기부 장관 “구미 전문인력 양성 돕겠다”,경제>경제일반,지역>울산,지역>대구,NaN,NaN,NaN,"이,이영","구미전자공업고등학교,구미,구미새마을중앙시장","구미시,중기부,상인회,중소벤처기업부,구미전자공고","이영,중기부,장관,구미,전문,인력,양성,전통시장,구미전자공고,차례,방문,현장,목소리...","이영,구미전자공고,구미,구미새마을중앙시장,전통시장,상인들,중기부,상인회,코로나19,...",이영 중소벤처기업부 장관이 지난 17일 구미를 방문했다. \n \n이 장관의 이번 ...,http://www.idaegu.com/newsView/idg202211200031,NaN
1,1.500601e+06,20221118,매일신문,이영광,"이헌승 국회 국방위원장, 구미 방위산업체 방문...'방산클러스터 유치' 기대감 쑥쑥",지역>경북,지역>대구,지역>광주,NaN,NaN,NaN,"이,이수영,구자근,어성철,이헌승","방산,경북,구미시의원,구미,구미국가산업단지,경북도의원","LIG넥스원,구미시,㈜한화,㈜LIG넥스원,구미,국회,국방위,국방위원회,한화시스템","국방위원장,이헌승,국회,국방,위원장,방문,구미,방위,산업체,방산클러스터,유치,기대감...","구미,이헌승,위원장,방산업체,한화시스템,넥스원,간담회,lig넥스원,구미시,구자근,국...",이헌승 국회 국방위원회 위원장이 18일 방산 투자 활기를 띠고 있는 경북 구미를 방...,https://news.imaeil.com/page/view/202211181822...,NaN
2,1.500601e+06,20221118,매일신문,이창희,경북 구미 강소연구개발특구 육성사업 중간보고 및 간담회,지역>경북,지역>충남,지역>대전,NaN,NaN,NaN,"곽호상,윤창배,김장호","강소특구,금오테크노밸리,경북,구미시,구미,경,경북지역본부장,78만평","과학기술정보통신부,한국산업단지공단,금오공대","중간,경북,구미,강소,특구,육성,사업,간담회,구미시,금오공대,한국산업단지공단,추진,...","구미,금오공대,한국산업단지공단,경북,김장호,활성화,강소,과학기술정보통신부,곽호상,5...","구미시, 금오공대, 한국산업단지공단은 18일 금오공대에서 '경북 구미 강소연구개발특...",https://news.imaeil.com/page/view/202211181440...,NaN
3,2.100801e+06,20221118,아시아경제,영남취재본부 김용우,경북 구미 ‘강소연구개발특구’ 육성사업 중간보고회,지역>경북,지역>충남,지역>대전,NaN,NaN,NaN,"곽호상,윤창배,김장호,강소연구개발특구","구미하이테크밸리,구미국가산업,금오테크노밸리,경북,전주,구미시,78여만평,구미,경북지...","과학기술정보통신부,금오공과대학교,구미시,국립금오공과대학교,한국산업단지공단,금오공대","강소연구개발특구,경북,구미,강소연구,특구,육성,사업,중간보,고회,구미시,금오공과대학...","경북,구미,한국산업단지공단,구미시,금오공과대학교,금오공대,김장호,구미하이테크밸리,사...",[아시아경제 영남취재본부 김용우 기자] 구미시는 18일 금오공과대학교에서 김장호 구...,https://view.asiae.co.kr/article/2022111813080...,NaN
4,1.500601e+06,20221118,매일신문,이창희,"순천향대 구미병원, '협력 병의원 만남의 장' 개최",정치>국회_정당,지역>울산,지역>대구,NaN,NaN,NaN,"위암,박준용,정일권","금오산,구미,구미시의사회장","구미병원,구미병원장,순천향대","순천향대,구미병원,만남,협력,개최,협력병의원,소통,화합,식견,공유,순천향대,구미병원...","구미,정일권,탈출증,연수강좌,협력병의원,구미병원장,구미병원,정일,신시옥,100여,소...",순천향대 구미병원은 17일 구미 호텔금오산에서 박준용 구미시의사회장과 협력 병 의원...,https://news.imaeil.com/page/view/202211181036...,NaN


전처리 파트

In [ ]:
# 뉴스기사 전처리
p = re.compile('[^ ㄱ-ㅣ가-힣+]')

def remove_special_characters(sentence):
    sentence = p.sub(' ',sentence)
    sentence = ' '.join(sentence.split())
    return sentence

df['전처리기사'] = df['본문'].map(remove_special_characters)
df['전처리기사'] = df['전처리기사'].map(lambda x: x.replace('대구', '대구시'))
df['전처리기사'] = df['전처리기사'].map(lambda x: x.replace('대구시시', '대구시'))

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
korean_stopwords_path = "/content/drive/MyDrive/구미시 뉴스기사 분류/data/korean_stopwords.txt"

with open(korean_stopwords_path, encoding = 'utf-8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

In [ ]:
def remove_stopwords(sentence):
    word_tokens = word_tokenize(sentence)
    word_list = []

    for word in word_tokens:
        if word not in stopwords:
            if word[:2] != '구미' and word[:2] != '경북':
                word_list.append(word)
        sentence = ' '.join(word_list)
    return sentence

df['불용어제거기사'] = df['전처리기사'].map(remove_stopwords)

Mecab과 SBERT를 이용한 Bertopic

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [ ]:
custom_tokenizer = CustomTokenizer(Mecab())

In [ ]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [ ]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics= 30,  # 모델이 자동으로 토픽의 수를 조정하도록 하려면 nr_topics = 'auto'로 설정
                 top_n_words=10,
                 calculate_probabilities=True)

In [ ]:
topics, probs = model.fit_transform(df['불용어제거기사'])

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
model.get_topic_info() # Name : top 4 word in that document

,Topic,Count,Name
0,-1,12155,-1_으로_에서_사업_지역
1,0,631,0_소방_화재_공장_당국
2,1,480,1_대구시_취수_낙동강_문제
3,2,428,2_대통령_박정희_생가_의원
4,3,393,3_아동_학교_어린이집_체험
5,4,358,4_투자_체결_협약_로봇
6,5,310,5_발견_숨진_경찰_친모
7,6,300,6_프린팅_참석_기업_산업
8,7,283,7_확진_코로나_신규_집계
9,8,258,8_번호_당첨_로또_복권


In [ ]:
model.visualize_topics()

In [ ]:
model.visualize_hierarchy(top_n_topics=30)

In [ ]:
model.visualize_barchart([i for i in range(12)], n_words = 7) #n_words = int --> topic 별 등장 단어 개수 조절 default : 5

In [ ]:
model.visualize_distribution(probs[423])

In [ ]:
for i in range(0, 30):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('소방', 0.08304932636598084), ('화재', 0.07460098799051512), ('공장', 0.06833218715734518), ('당국', 0.06758631521881303), ('진화', 0.06744331516848942), ('발생', 0.05626080698232676), ('피해', 0.04890274338676782), ('습니다', 0.04611323679496779), ('에서', 0.0435400239798542), ('사고', 0.04350810078438044)]
1 번째 토픽 : [('대구시', 0.06273540437938831), ('취수', 0.05770027875503796), ('낙동강', 0.05437664843236409), ('문제', 0.03949006398810807), ('대구', 0.0371736324981538), ('이전', 0.03553404448608834), ('환경부', 0.03227510486867241), ('수돗물', 0.024970583623117153), ('해결', 0.02291523591734885), ('공급', 0.022440645931063208)]
2 번째 토픽 : [('대통령', 0.06753905116329963), ('박정희', 0.06401697437206143), ('생가', 0.030985768314025695), ('의원', 0.030928282543175502), ('마을', 0.030030492345736362), ('추도식', 0.027142329156589904), ('시민', 0.02565120125620389), ('명칭', 0.025342278992249296), ('반대', 0.023643895423800954), ('역사', 0.022176494834585052)]
3 번째 토픽 : [('아동', 0.06747911264687832), ('학교', 0.04270888389074167), ('어린이집', 0.03